# Motion Matching For Character Animation

by Simon Clavet, Michael Buttner  
~2012

Notebook by Jerome Eippers, 2025

In [ ]:
%matplotlib widget
import numpy as np
from ipywidgets import widgets, interact
from matplotlib import pyplot as plt


import ipyanimlab as lab

viewer = lab.Viewer(move_speed=5, width=1280, height=720)

## Predicting Character Motion with a Spring–Damper Model

In motion matching, we often need to predict the character’s *future position* and *velocity* to find the best matching animation pose.
Instead of using a simple linear extrapolation, we can use a **critically damped spring–damper system**, which provides smooth,
physically plausible motion towards a goal position.

You can check the awesome article from Daniel Holden about springs that explains all the concepts. The code is taken from his page.

**https://theorangeduck.com/page/spring-roll-call**


---

Because the spring system models **how the character approaches a desired state (target position, orientation, or velocity)**,
it naturally predicts where the character *will be* after a short time step, assuming it continues trying to reach that goal.

- It incorporates **velocity** and **acceleration** history.
- It accounts for **smooth exponential decay** toward the goal.
- It produces **timestep-independent** predictions, unlike simple Euler integration.

Thus, by running the spring–damper update with the current goal pose, we can predict the *future position and velocity* of the character
for motion matching — providing a smooth, physically coherent estimate for comparing to animation clips.


In [ ]:
LN2 = np.log(2.0)
def fast_negexp(x):
    """Fast approximation of exp(-x) using rational function"""
    return 1.0 / (1.0 + x + 0.48*x*x + 0.235*x*x*x)
    
def halflife_to_damping(halflife, eps=1e-5):
    return (4.0 * LN2) / (halflife + eps)

def quat_abs(q):
    sign = np.where(q[..., 0] < 0, -1.0, 1.0)
    return q * sign[..., None]

def quat_log(x, eps=1e-8):
    length = np.sqrt(np.sum(np.square(x[...,1:]), axis=-1))[...,np.newaxis]
    halfangle = np.where(length < eps, np.ones_like(length), np.arctan2(length, x[...,0:1]) / (length+eps))
    return halfangle * x[...,1:]

def quat_exp(x, eps=1e-8):
    halfangle = np.sqrt(np.sum(np.square(x), axis=-1))[...,np.newaxis]
    c = np.where(halfangle < eps, np.ones_like(halfangle), np.cos(halfangle))
    s = np.where(halfangle < eps, np.ones_like(halfangle), np.sinc(halfangle / np.pi))
    return np.concatenate([c, s * x], axis=-1)

def quat_to_scaled_angle_axis(x, eps=1e-8):
    return 2.0 * quat_log(x, eps)

def quat_from_scaled_angle_axis(x, eps=1e-8):
    return quat_exp(x / 2.0, eps)

def simple_spring_damper_exact(x, v, x_goal, halflife, dt):
    y = halflife_to_damping(halflife) / 2.0
    j0 = x - x_goal
    j1 = v + j0 * y
    eydt = fast_negexp(y * dt)

    x_next = eydt * (j0 + j1 * dt) + x_goal
    v_next = eydt * (v - j1 * y * dt)

    return x_next, v_next

def simple_spring_damper_exact_quat(x, v, x_goal, halflife, dt):
    y = halflife_to_damping(halflife) / 2.0
    eydt = fast_negexp(y * dt)

    # relative rotation
    q_rel = quat_abs(lab.utils.quat_mul(x, lab.utils.quat_inv(x_goal)))

    j0 = quat_to_scaled_angle_axis(q_rel)
    j1 = v + j0 * y

    x_next = lab.utils.quat_mul(quat_from_scaled_angle_axis(eydt * (j0 + j1 * dt)), x_goal)
    v_next = eydt * (v - j1 * y * dt)
    return lab.utils.quat_normalize(x_next), v_next

def spring_character_update(x, v, a, v_goal, halflife, dt):
    y = halflife_to_damping(halflife) / 2.0
    j0 = v - v_goal
    j1 = a + j0 * y
    eydt = fast_negexp(y * dt)

    x_next = (
        eydt * ((-j1) / (y * y) + (-j0 - j1 * dt) / y)
        + (j1 / (y * y))
        + (j0 / y)
        + v_goal * dt
        + x
    )
    v_next = eydt * (j0 + j1 * dt) + v_goal
    a_next = eydt * (a - j1 * y * dt)

    return x_next, v_next, a_next


In [ ]:
gamepad = widgets.Controller(index=0)
gamepad

In [ ]:
# create a sphere to display in our 3d viewport

def generate_sphere_points(radius=1.0, segments=16):
    theta = np.linspace(0, 2*np.pi, segments)[:, None]  
    phi = np.linspace(0, 2*np.pi, segments)[None, :]
    
    # Spherical coordinates to Cartesian
    x_lat = radius * np.cos(theta) * np.cos(phi)
    y_lat = radius * np.sin(theta)
    z_lat = radius * np.cos(theta) * np.sin(phi)
    
    lat_points = np.stack([x_lat, y_lat.repeat(segments, axis=1), z_lat], axis=-1).reshape(-1, 3)
    
    x_lon = radius * np.cos(phi) * np.cos(theta)
    y_lon = radius * np.sin(phi)
    z_lon = radius * np.cos(phi) * np.sin(theta)
    
    lon_points = np.stack([x_lon, y_lon.repeat(segments, axis=0), z_lon], axis=-1).reshape(-1, 3)
    
    # Combine all points
    points = np.concatenate([lat_points, lon_points], axis=0)
    
    return np.array(points, dtype=np.float32)

sphere_lines = generate_sphere_points(10, 9)

### Simulate the character motion
by using the spring character update we can move our character and predict the future positions and orientations !

In [ ]:
x = np.zeros(3, dtype=np.float32)
v = np.zeros(3, dtype=np.float32)
a = np.zeros(3, dtype=np.float32)

x_rot = np.array([1,0,0,0], dtype=np.float32)
v_rot = np.zeros(3, dtype=np.float32)

desired_orientation = np.array([1,0,0,0], dtype=np.float32)

def render(frame, max_speed=3.5, halflife=.3, halflife_rot=.3):

    controller_orient = np.array([1,0,0,0], dtype=np.float32)
    posx = gamepad.axes[0].value 
    posz = -gamepad.axes[1].value 
    magnitude = 0
    if np.abs(posx) > 0.001 or np.abs(posz) > 0.001:
        angle = np.atan2(posz, posx)
        controller_orient[0] = np.cos(angle/2)
        controller_orient[2] = np.sin(angle/2)
        desired_orientation[:] = controller_orient
        magnitude = max_speed * 100

    posx = gamepad.axes[2].value 
    posz = -gamepad.axes[3].value 
    if np.abs(posx) > 0.001 or np.abs(posz) > 0.001:
        angle = np.atan2(posz, posx)
        desired_orientation[0] = np.cos(angle/2)
        desired_orientation[2] = np.sin(angle/2)

    desired_velocity = lab.utils.quat_mul_vec(controller_orient, np.array([0,0,1], dtype=np.float32))
    desired_velocity *= magnitude

    x[:], v[:], a[:] = spring_character_update(x,v,a, desired_velocity, halflife, 1.0/30.)
    x_rot[:], v_rot[:] = simple_spring_damper_exact_quat(x_rot, v_rot, desired_orientation, halflife_rot, 1.0/30.)
    
    viewer.begin_shadow()
    viewer.end_shadow()
    
    viewer.begin_display()
    viewer.draw_ground()
    viewer.end_display()

    #viewer.disable(depth_test=True)

    lines = sphere_lines + x[np.newaxis, :]
    viewer.draw_lines(lines)
    viewer.draw_lines(np.array([np.zeros(3, dtype=np.float32), lab.utils.quat_mul_vec(x_rot, np.array([0,0,100]))], dtype=np.float32)+x[np.newaxis, :].astype(np.float32))
    
    future_x, _, _ = spring_character_update(x,v,a, desired_velocity, halflife, 10./30.)
    lines = sphere_lines + future_x[np.newaxis, :].astype(np.float32)
    viewer.draw_lines(lines, color=np.array([1,1,0], dtype=np.float32))
    future_rot_x, _ = simple_spring_damper_exact_quat(x_rot, v_rot, desired_orientation, halflife_rot, 10.0/30.)
    viewer.draw_lines(np.array([np.zeros(3, dtype=np.float32), lab.utils.quat_mul_vec(future_rot_x, np.array([0,0,100]))], dtype=np.float32)+future_x[np.newaxis, :].astype(np.float32).astype(np.float32), color=np.array([1,1,0], dtype=np.float32))

    future_x, _, _ = spring_character_update(x,v,a, desired_velocity, halflife, 20./30.)
    lines = sphere_lines + future_x[np.newaxis, :].astype(np.float32)
    viewer.draw_lines(lines, color=np.array([1,1,0], dtype=np.float32))
    future_rot_x, _ = simple_spring_damper_exact_quat(x_rot, v_rot, desired_orientation, halflife_rot, 20.0/30.)
    viewer.draw_lines(np.array([np.zeros(3, dtype=np.float32), lab.utils.quat_mul_vec(future_rot_x, np.array([0,0,100]))], dtype=np.float32)+future_x[np.newaxis, :].astype(np.float32).astype(np.float32), color=np.array([1,1,0], dtype=np.float32))

    future_x, _, _ = spring_character_update(x,v,a, desired_velocity, halflife, 30./30.)
    lines = sphere_lines + future_x[np.newaxis, :].astype(np.float32)
    viewer.draw_lines(lines, color=np.array([1,1,0], dtype=np.float32))
    future_rot_x, _ = simple_spring_damper_exact_quat(x_rot, v_rot, desired_orientation, halflife_rot, 30.0/30.)
    viewer.draw_lines(np.array([np.zeros(3, dtype=np.float32), lab.utils.quat_mul_vec(future_rot_x, np.array([0,0,100]))], dtype=np.float32)+future_x[np.newaxis, :].astype(np.float32).astype(np.float32), color=np.array([1,1,0], dtype=np.float32))

    viewer.execute_commands()
    
interact(
    render, 
    frame=lab.Timeline(max=100),
)
viewer

## Load Character and Animations

In [ ]:
# load the character
character = viewer.import_usd_asset('AnimLabSimpleMale.usd')

In [ ]:
displacement_asset = viewer.import_usd_asset('../../meshes/displacement.usd')

In [ ]:
animmap = lab.AnimMapper(character, keep_translation=False, root_motion=True, match_effectors=True, local_offsets={'Hips':[0, 2, 0]})
animations = []

animations.append(lab.import_bvh(f'../../resources/lafan1/bvh/run2_subject4.bvh', anim_mapper=animmap))
animations.append(lab.import_bvh(f'../../resources/lafan1/bvh/run1_subject2.bvh', anim_mapper=animmap))
animations.append(lab.import_bvh(f'../../resources/lafan1/bvh/run1_subject5.bvh', anim_mapper=animmap))
animations.append(lab.import_bvh(f'../../resources/lafan1/bvh/run2_subject1.bvh', anim_mapper=animmap))

bone_count = character.bone_count()
bones = animations[0].bones
parents = animations[0].parents

In [ ]:
def render(frame, index=0):

    frame = min(frame, animations[index].quats.shape[0] -1)
    q = (animations[index].quats[frame,...])
    p = (animations[index].pos[frame,...])
        
    a =  lab.utils.quat_to_mat(q, p)
    viewer.set_shadow_poi(p[0])
    
    viewer.begin_shadow()
    viewer.draw(character, a)
    viewer.end_shadow()
    
    viewer.begin_display()
    viewer.draw_ground()
    viewer.draw(character, a)
    
    viewer.end_display()

    viewer.disable(depth_test=True)
   
    viewer.draw_axis(character.world_skeleton_xforms(a), 5)
    viewer.draw_lines(character.world_skeleton_lines(a))
    
    viewer.execute_commands()
    
interact(
    render, 
    frame=lab.Timeline(max=8000),
    index=widgets.IntSlider(max=len(animations)-1)
)
viewer

### Stabilizing Displacement and Orientation with Savitzky–Golay Filter

In motion matching, the displacement and orientation of the character are critical for selecting appropriate motion clips. However, raw hip trajectories can be noisy.

To reduce this noise while preserving the overall motion trend, we apply a **Savitzky–Golay filter** to the projected hip (root) positions:

$$
\mathbf{h}_t^\text{smoothed} = \text{SG}(\mathbf{h}_t, \text{window}, \text{poly\_order})
$$

- Here, $\mathbf{h}_t$ is the root position at time $t$.  
- $\text{SG}(\cdot)$ denotes the Savitzky–Golay smoothing operation.  
- `window` is the size of the smoothing window, and `poly_order` is the polynomial order used for local fitting.  

The Savitzky–Golay filter effectively smooths high-frequency noise while preserving the **velocity and curvature** of the motion. This is important because:

1. It stabilizes the **displacement** used in distance metrics for motion matching.  
2. It prevents abrupt changes in **orientation**, which could lead to unrealistic transitions.  

After smoothing, the filtered trajectory better reflects the intended motion, improving the reliability of motion matching calculations.


In [ ]:
from scipy.signal import savgol_filter

def prepare_data(quats, pos, last_range_value=0):
    # copy this animation into new buffers
    bone_quats = quats.copy()
    bone_pos = pos.copy()

    # compute range
    frame_count = bone_quats.shape[0]
    frame_ranges = np.ones([frame_count, 2], dtype=np.uint32)
    frame_ranges[:, 0] = last_range_value
    frame_ranges[:, 1] = (last_range_value + frame_count)

    # smooth the root
    root_x = savgol_filter(bone_pos[:, 0, 0], 60, 4)
    root_y = savgol_filter(bone_pos[:, 0, 1], 60, 4)
    root_z = savgol_filter(bone_pos[:, 0, 2], 60, 4)
    root_p = np.stack([root_x, root_y, root_z]).T
    
    theta =  np.atan2(
        2 * bone_quats[:, 0, 0] * bone_quats[:, 0, 2], 
        1.0 - (2 * bone_quats[:, 0, 2] * bone_quats[:, 0, 2])
    )
    theta = savgol_filter(np.unwrap(theta), 65, 5)
    root_q = np.zeros([frame_count, 4], dtype=np.float32)
    root_q[:, 0] = np.cos(theta/2.)
    root_q[:, 2] = np.sin(theta/2.)

    # align the data to the filtered root
    gq, gp = lab.utils.quat_fk(bone_quats, bone_pos, parents)
    gq[:, 0, :] = root_q
    gp[:, 0, :] = root_p
    bone_quats, bone_pos = lab.utils.quat_ik(gq, gp, parents)

    # Compute velocities via central difference
    velocities = np.empty_like(bone_pos)
    velocities[1:-1] = (
        0.5 * (bone_pos[2:  ] - bone_pos[1:-1]) * 30.0 +
        0.5 * (bone_pos[1:-1] - bone_pos[ :-2]) * 30.0)
    bone_pos[ 0] = bone_pos[ 1] - (bone_pos[ 3] - bone_pos[ 2])
    bone_pos[-1] = bone_pos[-2] + (bone_pos[-2] - bone_pos[-3])
    
    # Same for angular velocities
    angular_velocities = np.zeros_like(bone_pos)
    angular_velocities[1:-1] = (
        0.5 * quat_to_scaled_angle_axis(quat_abs(lab.utils.quat_mul(bone_quats[2:  ], lab.utils.quat_inv(bone_quats[1:-1])))) * 30.0 +
        0.5 * quat_to_scaled_angle_axis(quat_abs(lab.utils.quat_mul(bone_quats[1:-1], lab.utils.quat_inv(bone_quats[ :-2])))) * 30.0)
    angular_velocities[ 0] = angular_velocities[ 1] - (angular_velocities[ 3] - angular_velocities[ 2])
    angular_velocities[-1] = angular_velocities[-2] + (angular_velocities[-2] - angular_velocities[-3])

    return bone_quats, bone_pos, velocities, angular_velocities, frame_ranges, frame_count
    

In [ ]:
def build_database(slices):
    frame_count = 0
    bone_rotations = []
    bone_positions = [] 
    bone_velocities = []
    bone_angular_velocities = []
    frame_ranges = []
    
    for (anim_id, frame_slice) in slices:
        q, p, v, a, r, c = prepare_data(animations[anim_id].quats[frame_slice], animations[anim_id].pos[frame_slice], frame_count)
        frame_count += c
        bone_rotations.append(q)
        bone_positions.append(p)
        bone_velocities.append(v)
        bone_angular_velocities.append(a)
        frame_ranges.append(r)

    return (
        np.concatenate(bone_rotations),
        np.concatenate(bone_positions),
        np.concatenate(bone_velocities),
        np.concatenate(bone_angular_velocities),
        np.concatenate(frame_ranges),
        frame_count
    )
    
bone_rotations, bone_positions, bone_velocities, bone_angular_velocities, frame_ranges, frame_count = build_database([
    (0, slice(80, 4300)),
    (1, slice(105, 7000)),
    (2, slice(90, 7000)),
])

In [ ]:
def render(frame):

    q = bone_rotations[frame]
    p = bone_positions[frame]
        
    a =  lab.utils.quat_to_mat(q, p)
    viewer.set_shadow_poi(p[0])
    
    viewer.begin_shadow()
    viewer.draw(character, a)
    viewer.end_shadow()
    
    viewer.begin_display()
    viewer.draw_ground()
    viewer.draw(character, a)
    
    d = lab.utils.quat_to_mat(q[0], p[0])
    viewer.draw(displacement_asset, d)
    
    viewer.end_display()

    viewer.disable(depth_test=True)
   
    viewer.draw_axis(character.world_skeleton_xforms(a), 5)
    viewer.draw_lines(character.world_skeleton_lines(a))
    
    viewer.execute_commands()
    
interact(
    render, 
    frame=lab.Timeline(max = frame_count-1)
)
viewer

### Building the Feature Vector for Motion Matching

To perform motion matching, we construct a feature vector that captures both the current state and predicted future motion. This feature vector includes:

1. **Hips velocity** - keep character momentum.
2. **Feet positions and velocities** – to capture the foot dynamics.   
3. **Future states** – sampled at 10, 20, and 30 frames ahead to anticipate motion.

Formally, the feature vector $\mathbf{f}_t$ at time $t$ can be written as:

$$
\mathbf{f}_t = 
\Big[
\mathbf{v}_\text{hips}(t),
\mathbf{p}_\text{feet}(t), \mathbf{v}_\text{feet}(t), 
\mathbf{p}_\text{char}(t+10), 
\mathbf{p}_\text{char}(t+20), 
\mathbf{p}_\text{char}(t+30), 
\mathbf{o}_\text{char}(t+10), 
\mathbf{o}_\text{char}(t+20), 
\mathbf{o}_\text{char}(t+30)
\Big]
$$

- $\mathbf{v}_\text{hips}(t)$: velocity of the hips at time $t$.  
- $\mathbf{p}_\text{feet}(t)$: positions of the left and right feet at time $t$.  
- $\mathbf{v}_\text{feet}(t)$: velocities of the feet at time $t$.  
- $\mathbf{p}_\text{char}(t+\Delta)$, $\mathbf{o}_\text{char}(t+\Delta)$: predicted future position and orientation at frame offsets $\Delta = 10, 20, 30$.  

By including these future states in the feature vector, motion matching can select candidate clips that not only match the current pose but also align with the intended future motion. This helps achieve **smoother transitions and realistic animation trajectories**.


In [ ]:
features = np.zeros([frame_count, 33], np.float32)

gq, gp = lab.utils.quat_fk(bone_rotations, bone_positions, parents)
lfoot_id = bones.index('LeftFoot')
rfoot_id = bones.index('RightFoot')
hips_id = bones.index('Hips')

for f in range(frame_count):
    f_indices = np.minimum(np.array([0, 1, 10, 20, 30], dtype=np.int16) + f, frame_ranges[f, 1]-1)

    # get the data for the features
    gather_p = gp[np.ix_(f_indices, [0,hips_id,lfoot_id,rfoot_id])]
    gather_q = gq[np.ix_(f_indices, [0,hips_id,lfoot_id,rfoot_id])]

    # convert the world data into local space of the current frame
    gather_q, gather_p = lab.utils.qp_mul( lab.utils.qp_inv((gather_q[0:1, 0], gather_p[0:1, 0])), (gather_q, gather_p))
    gather_root_dir = lab.utils.quat_mul_vec(gather_q[:, 0], np.array([0,0,1], dtype=np.float32))

    # take the features
    features[f, 0:3] = gather_p[1, 1] - gather_p[0, 1] # Hips velocity
    features[f, 3:6] = gather_p[0, 2] # Left foot position
    features[f, 6:9] = gather_p[0, 3] # Right foot position
    features[f, 9:12] = gather_p[1, 2] - gather_p[0, 2] # Left foot velocity
    features[f, 12:15] = gather_p[1, 3] - gather_p[0, 3] # right foot velocity
    features[f, 15:18] = gather_p[2, 0] # future trajectory position
    features[f, 18:21] = gather_p[3, 0] # future trajectory position
    features[f, 21:24] = gather_p[4, 0] # future trajectory position
    features[f, 24:27] = gather_root_dir[2] # future trajectory direction
    features[f, 27:30] = gather_root_dir[3] # future trajectory direction
    features[f, 30:33] = gather_root_dir[4] # future trajectory direction

In [ ]:
def render(frame):

    q = bone_rotations[frame].copy()
    p = bone_positions[frame].copy()

    q[0] = np.array([1,0,0,0])
    p[0] = 0
        
    a =  lab.utils.quat_to_mat(q, p)
    viewer.set_shadow_poi(p[0])
    
    viewer.begin_shadow()
    viewer.draw(character, a)
    viewer.end_shadow()
    
    viewer.begin_display()
    viewer.draw_ground()
    viewer.draw(character, a)
    viewer.end_display()

    lines = sphere_lines + features[frame, 3:6]
    viewer.draw_lines(lines)
    lines = sphere_lines + features[frame, 6:9]
    viewer.draw_lines(lines)

    lines = sphere_lines + features[frame, 15:18]
    viewer.draw_lines(lines)
    lines = sphere_lines + features[frame, 18:21]
    viewer.draw_lines(lines)
    lines = sphere_lines + features[frame, 21:24]
    viewer.draw_lines(lines)

    gq, gp = lab.utils.quat_fk(q, p, parents)

    viewer.draw_lines(np.asarray([features[frame, 15:18], features[frame, 15:18]+features[frame, 24:27]*50]))
    viewer.draw_lines(np.asarray([features[frame, 18:21], features[frame, 18:21]+features[frame, 27:30]*50]))
    viewer.draw_lines(np.asarray([features[frame, 21:24], features[frame, 21:24]+features[frame, 30:33]*50]))

    viewer.draw_lines(np.asarray([gp[hips_id], gp[hips_id]+features[frame, 0:3]*5]))
    viewer.draw_lines(np.asarray([gp[lfoot_id], gp[lfoot_id]+features[frame, 9:12]*5]))
    viewer.draw_lines(np.asarray([gp[rfoot_id], gp[rfoot_id]+features[frame, 12:15]*5]))

    viewer.disable(depth_test=True)
   
    #viewer.draw_axis(character.world_skeleton_xforms(a), 5)
    #viewer.draw_lines(character.world_skeleton_lines(a))

    viewer.execute_commands()
    
interact(
    render, 
    frame=lab.Timeline(max = frame_count-1)
)
viewer

### Feature Normalization

In an Euclidean search, we compute the distance between a query vector and all feature vectors in our dataset:

$$
d(\mathbf{x}, \mathbf{y}) = \sqrt{\sum_i (x_i - y_i)^2}
$$

If features have different scales, some features will dominate the distance. To prevent this, we normalize each feature using its mean $\mu_i$ and standard deviation $\sigma_i$:

$$
x_i^\text{norm} = \frac{x_i - \mu_i}{\sigma_i}
$$

After normalization, each feature has zero mean and unit variance:

$$
\text{mean}(x_i^\text{norm}) = 0, \quad \text{std}(x_i^\text{norm}) = 1
$$

This ensures that all features contribute equally to the Euclidean distance:

$$
d(\mathbf{x}^\text{norm}, \mathbf{y}^\text{norm}) = \sqrt{\sum_i (x_i^\text{norm} - y_i^\text{norm})^2}
$$

Without normalization, features with larger numeric ranges could dominate the distance, leading to biased results in the search.


In [ ]:
features_mean = features.mean(axis=0)
features_std = features.std(axis=0) + 1e-8
    
features_normalized = (features - features_mean)/features_std

## Player

In [ ]:

def inertialize_transition_vec3(off_x, off_v, src_x, src_v, dst_x, dst_v):
    off_x = (src_x + off_x) - dst_x
    off_v = (src_v + off_v) - dst_v
    return off_x, off_v


def inertialize_update_vec3(in_x, in_v, off_x, off_v, halflife, dt):
    off_x, off_v = simple_spring_damper_exact(off_x, off_v, np.zeros(3), halflife, dt)
    out_x = in_x + off_x
    out_v = in_v + off_v
    return out_x, out_v, off_x, off_v


def inertialize_transition_quat(off_x, off_v, src_x, src_v, dst_x, dst_v):
    off_x = quat_abs(lab.utils.quat_mul(lab.utils.quat_mul(off_x, src_x), lab.utils.quat_inv(dst_x)))
    off_v = (off_v + src_v) - dst_v
    return off_x, off_v


def inertialize_update_quat(in_x, in_v, off_x, off_v, halflife, dt):
    off_x, off_v = simple_spring_damper_exact_quat(off_x, off_v, np.array([1.,0,0,0]), halflife, dt)
    out_x = lab.utils.quat_mul(off_x, in_x)
    out_v = off_v +lab.utils.quat_mul_vec(off_x, in_v)
    return out_x, out_v, off_x, off_v

class Player:
    def __init__(self, frame):
        self.frame = frame
        self.q = bone_rotations[frame].copy()
        self.p = bone_positions[frame].copy()
        self.q[0] = np.array([1,0,0,0], dtype=np.float32)
        self.p[0] = np.zeros(3, dtype=np.float32)

        self.off_x = np.zeros([bone_count-1, 3], dtype=np.float32)
        self.off_v = np.zeros([bone_count-1, 3], dtype=np.float32)
        self.off_qx = np.zeros([bone_count-1, 4], dtype=np.float32)
        self.off_qx[:, 0] = 1
        self.off_qv = np.zeros([bone_count-1, 3], dtype=np.float32)

    def set_next_frame(self, frame, inertialize=True):

        # compute the frame of animation, by computing the local root displacement and adding it back to the current entity position.
        q = bone_rotations[frame].copy()
        p = bone_positions[frame].copy()
        prv_frame = int(max(frame-1, frame_ranges[frame, 0]))
        rq, rp = lab.utils.qp_mul(lab.utils.qp_inv((bone_rotations[prv_frame,0], bone_positions[prv_frame, 0])), (q[0], p[0]))
        q[0], p[0] = lab.utils.qp_mul((self.q[0], self.p[0]), (rq, rp))

        # do we blend the animation or just play back to back
        if inertialize:
            if frame != self.frame+1:
                self.off_x, self.off_v = inertialize_transition_vec3(self.off_x, self.off_v, bone_positions[self.frame, 1:], bone_velocities[self.frame, 1:], bone_positions[frame, 1:], bone_velocities[frame, 1:])
                self.off_qx, self.off_qv = inertialize_transition_quat(self.off_qx, self.off_qv, bone_rotations[self.frame, 1:], bone_angular_velocities[self.frame, 1:], bone_rotations[frame, 1:], bone_angular_velocities[frame, 1:])
    
            p[1:], _, self.off_x, self.off_v = inertialize_update_vec3(bone_positions[frame, 1:], bone_velocities[frame, 1:], self.off_x, self.off_v, .09, 1.0/30.)
            q[1:], _, self.off_qx, self.off_qv = inertialize_update_quat(bone_rotations[frame, 1:], bone_angular_velocities[frame, 1:], self.off_qx, self.off_qv, .09, 1.0/30.)

        # store the current frame of animation
        self.frame = frame
        self.q, self.p = q, p

In [ ]:
x = np.zeros(3, dtype=np.float32)
v = np.zeros(3, dtype=np.float32)
a = np.zeros(3, dtype=np.float32)

player = Player(0)

x_rot = np.array([1,0,0,0], dtype=np.float32)
v_rot = np.zeros(3, dtype=np.float32)

desired_orientation = np.array([1,0,0,0], dtype=np.float32)

def render(frame, max_speed=4., halflife=.5, halflife_rot=.4, code_vs_anim=0., fast_stop=False, inertialize=False):

    controller_orient = np.array([1,0,0,0], dtype=np.float32)
    posx = gamepad.axes[0].value 
    posz = -gamepad.axes[1].value 
    magnitude = 0
    if np.abs(posx) > 0.001 or np.abs(posz) > 0.001:
        angle = np.atan2(posz, posx)
        controller_orient[0] = np.cos(angle/2)
        controller_orient[2] = np.sin(angle/2)
        desired_orientation[:] = controller_orient
        magnitude = max_speed * 100 * np.sqrt(posx*posx + posz*posz)
    elif fast_stop:
        halflife *= 0.5

    posx = gamepad.axes[2].value 
    posz = -gamepad.axes[3].value 
    if np.abs(posx) > 0.001 or np.abs(posz) > 0.001:
        angle = np.atan2(posz, posx)
        desired_orientation[0] = np.cos(angle/2)
        desired_orientation[2] = np.sin(angle/2)

    desired_velocity = lab.utils.quat_mul_vec(controller_orient, np.array([0,0,1], dtype=np.float32))
    desired_velocity *= magnitude

    # Compute future trajectory
    x[:], v[:], a[:] = spring_character_update(x,v,a, desired_velocity, halflife, 1.0/30.)
    x_rot[:], v_rot[:] = simple_spring_damper_exact_quat(x_rot, v_rot, desired_orientation, halflife_rot, 1.0/30.)
    future_x_1, _, _ = spring_character_update(x,v,a, desired_velocity, halflife, 10./30.)
    future_x_2, _, _ = spring_character_update(x,v,a, desired_velocity, halflife, 20./30.)
    future_x_3, _, _ = spring_character_update(x,v,a, desired_velocity, halflife, 30./30.)
    future_r_1, _ = simple_spring_damper_exact_quat(x_rot, v_rot, desired_orientation, halflife_rot, 10.0/30.)
    future_r_2, _ = simple_spring_damper_exact_quat(x_rot, v_rot, desired_orientation, halflife_rot, 20.0/30.)
    future_r_3, _ = simple_spring_damper_exact_quat(x_rot, v_rot, desired_orientation, halflife_rot, 30.0/30.)

    f1q, f1p = lab.utils.qp_mul(lab.utils.qp_inv((player.q[0], player.p[0])), (future_r_1, future_x_1))
    f2q, f2p = lab.utils.qp_mul(lab.utils.qp_inv((player.q[0], player.p[0])), (future_r_2, future_x_2))
    f3q, f3p = lab.utils.qp_mul(lab.utils.qp_inv((player.q[0], player.p[0])), (future_r_3, future_x_3))

    # Build the query
    query_vector = features_normalized[player.frame, :].copy()
    query_vector[15:18] = f1p # future trajectory position
    query_vector[18:21] = f2p # future trajectory position
    query_vector[21:24] = f3p # future trajectory position
    query_vector[24:27] = lab.utils.quat_mul_vec(f1q, np.array([0,0,1])) # future trajectory direction
    query_vector[27:30] = lab.utils.quat_mul_vec(f2q, np.array([0,0,1])) # future trajectory direction
    query_vector[30:33] = lab.utils.quat_mul_vec(f3q, np.array([0,0,1])) # future trajectory direction
    query_vector[15:33] = (query_vector[15:33] - features_mean[15:33])/features_std[15:33]

    # Search the DataBase
    ordered_frames = np.argsort(np.sum((features_normalized-query_vector)**2, axis=1))
    best_frame = ordered_frames[0]
    for i in range(50):
        best_frame = ordered_frames[i]
        # if we don't rewind for less than 5 frames
        if best_frame<player.frame-5 or best_frame >= player.frame:
            # if we are not 30 frames from the end of the animation
            if best_frame < frame_ranges[best_frame, 1]-30:
                break


    # compute character animation
    player.set_next_frame(best_frame+1, inertialize)
    q, p = player.q, player.p

    # force simulation to align to anim
    x[:] = (1.0-code_vs_anim) * p[0] + code_vs_anim * x[:]
    p[0] = x[:]
    
    m =  lab.utils.quat_to_mat(q, p)
    viewer.set_shadow_poi(p[0])
    
    viewer.begin_shadow()
    viewer.draw(character, m)
    viewer.end_shadow()
    
    viewer.begin_display()
    viewer.draw_ground()
    viewer.draw(character, m)
    viewer.end_display()

    #viewer.disable(depth_test=True)

    lines = sphere_lines + x[np.newaxis, :]
    viewer.draw_lines(lines, color=np.array([1,0,0], dtype=np.float32))
    viewer.draw_lines(np.array([[0,0,0], lab.utils.quat_mul_vec(x_rot, np.array([0,0,50]))], dtype=np.float32) + x[np.newaxis, :], color=np.array([1,0,0], dtype=np.float32))
    
    lines = sphere_lines + future_x_1[np.newaxis, :].astype(np.float32)
    viewer.draw_lines(lines, color=np.array([1,1,0], dtype=np.float32))
    viewer.draw_lines(np.array([[0,0,0], lab.utils.quat_mul_vec(future_r_1, np.array([0,0,50]))], dtype=np.float32) + future_x_1[np.newaxis, :].astype(np.float32), color=np.array([1,1,0], dtype=np.float32))

    lines = sphere_lines + future_x_2[np.newaxis, :].astype(np.float32)
    viewer.draw_lines(lines, color=np.array([1,1,0], dtype=np.float32))
    viewer.draw_lines(np.array([[0,0,0], lab.utils.quat_mul_vec(future_r_2, np.array([0,0,50]))], dtype=np.float32) + future_x_2[np.newaxis, :].astype(np.float32), color=np.array([1,1,0], dtype=np.float32))

    lines = sphere_lines + future_x_3[np.newaxis, :].astype(np.float32)
    viewer.draw_lines(lines, color=np.array([1,1,0], dtype=np.float32))
    viewer.draw_lines(np.array([[0,0,0], lab.utils.quat_mul_vec(future_r_3, np.array([0,0,50]))], dtype=np.float32) + future_x_3[np.newaxis, :].astype(np.float32), color=np.array([1,1,0], dtype=np.float32))


    lines = sphere_lines + lab.utils.quat_mul_vec(q[0], features[best_frame, 15:18]) + p[0]
    viewer.draw_lines(lines)
    lines = sphere_lines + lab.utils.quat_mul_vec(q[0], features[best_frame, 18:21]) + p[0]
    viewer.draw_lines(lines)
    lines = sphere_lines + lab.utils.quat_mul_vec(q[0], features[best_frame, 21:24]) + p[0]
    viewer.draw_lines(lines)

    viewer.execute_commands()
    
interact(
    render, 
    frame=lab.Timeline(max=100),
)
viewer

### Feature Weighting via Scaled Standard Deviation

To efficiently weight features during motion matching, we can incorporate the weight $w_i$ directly into the standard deviation used for normalization. Instead of:

$$
f_i^\text{scaled} = w_i \frac{f_i - \mu_i}{\sigma_i}
$$

we define an **adjusted standard deviation**:

$$
\sigma_i^\text{adj} = \frac{\sigma_i}{w_i}
$$

Then, the weighted and normalized feature becomes:

$$
f_i^\text{weighted} = \frac{f_i - \mu_i}{\sigma_i^\text{adj}} = \frac{f_i - \mu_i}{\sigma_i / w_i}
$$

This approach has two advantages:  

1. **Query efficiency** – at runtime, we only need a standard normalization $(f_i - μ_i) / σ_i^\text{adj}$ without multiplying by $w_i$.  
2. **Same effect as explicit weighting** – features with higher $w_i$ contribute more to the Euclidean distance, while smaller $w_i$ contribute less:

$$
d(\mathbf{f}, \mathbf{g}) = \sqrt{\sum_i \big(f_i^\text{weighted} - g_i^\text{weighted}\big)^2}
$$


In [ ]:
feature_weights = np.array([
    1, # Hips velocity
    .75, # Left foot position
    .75, # Right foot position
    1, # Left foot velocity
    1, # right foot velocity
    1 * .99**10, # future trajectory position
    1 * .99**20, # future trajectory position
    1 * .99**30, # future trajectory position
    1.5 * .99**10, # future trajectory direction
    1.5 * .99**20, # future trajectory direction
    1.5 * .99**30, # future trajectory direction
]).repeat(3)

# scale the features_std
features_std /= feature_weights

# re compute the normalized features
features_normalized = (features - features_mean)/features_std